In [1]:
import pandas as pd
import requests
import googlemaps
from datetime import datetime

In [2]:
df = pd.read_csv("../Mid-Term-Project/raw_data/traffic/traffic.csv")

df = df[df["Date"].str.contains("2020") == True]
df['traffic'] = (df.iloc[:, 5:].sum(axis=1)) / 24
df = df[['ID', 'Roadway Name', 'To', 'traffic']].copy()

/tmp/ipykernel_5067/837621976.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df['traffic'] = (df.iloc[:, 5:].sum(axis=1)) / 24


In [3]:
df_mean = df.groupby(['ID']).mean()
df_mean = df_mean.reset_index()

In [4]:
df = df.drop(['traffic'], axis=1)
df = df.drop_duplicates()

In [5]:
traffic = df.set_index('ID').join(df_mean.set_index('ID'))
traffic = traffic.reset_index()
traffic = traffic.drop_duplicates(subset=['ID'], keep='first')

In [6]:
gmaps = googlemaps.Client(key='GOOGLE_API_KEY')

In [ ]:
lat_long = []
for i in range(len(traffic)):
    first = traffic.iloc[i][1]
    second = traffic.iloc[i][2]
    geocode_result = gmaps.geocode(f'{first}, {second}, NY')
    coords = (geocode_result[0]['geometry']['location']['lat'], geocode_result[0]['geometry']['location']['lng'])
    lat_long.append(coords)

In [ ]:
lat = []
lng = []
for i in range(349):
    lat.append(lat_long[i][0])
    lng.append(lat_long[i][1])

In [ ]:
traffic = traffic.drop(['Roadway Name', 'To'], axis=1)
traffic['latitude'] = lat
traffic['longitude'] = lng

In [ ]:
traffic.to_csv('../Mid-Term-Project/processed_data/geocoded_traffic.csv', index=False, encoding='utf-8')